# Vifor - Accumulations
* Total count not exported

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("disclosure-report-switzerland-2019.pdf", pages='all', lattice=True)

## Format Table

In [3]:
df_export = df.copy()

#Shift
column = df_export.columns[0]
df_export[df_export[column].isna()] = df_export[df_export[column].isna()].shift(-1, axis='columns')

#Drop Columns
df_export = df_export.drop(df_export.columns[9:],axis=1)
df_export = df_export.drop(df_export.columns[[7]],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount', na=False)]
recipients = df_export[df_export.type.str.contains('Number of Recipients', na=False)]
rnd = df_export[df_export.type.str.contains('Transfers of Value', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Cleanup Numbers
df_export = cleanup_number(df_export)
df_export = remove_in_numbers(df_export, '-')
df_export = remove_comma(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"
df_export.loc[df_export.type == 'rnd', "total"] = df_export['donations_grants']
df_export.loc[df_export.type == 'rnd', "donations_grants"] = np.NaN

#Export
export_acumulations(df_export, 'vifor')

saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)

In [4]:
df_export

,type,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,source
0,hcp_amount,NaN,0.0,8044.98,15848.79,46800.0,1847.76,72541.53,vifor
1,hco_amount,7500.0,20052.7,4300.00,0.00,0.0,0.00,31852.70,vifor
2,hcp_count,NaN,NaN,20.00,18.00,19.0,1.00,NaN,vifor
3,hco_count,3.0,11.0,2.00,NaN,NaN,NaN,NaN,vifor
4,rnd,NaN,NaN,NaN,NaN,NaN,NaN,548960.62,vifor
